In [40]:
import pandas as pd
import numpy as np
import math
from datetime import date
from datetime import datetime

# set file name  - Confirmed, Deaths, Recovered
# set country of interest

In [26]:
filename = 'Confirmed'
country = 'US'

In [27]:
dt = datetime.now().strftime('%Y%m%d')

Covid 19 cases data from CSSE

In [28]:
df = pd.read_csv('covid_data/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-' + filename + '.csv')

In [29]:
df.shape

(450, 58)

In [30]:
df['Country/Region'].unique()

array(['Thailand', 'Japan', 'Singapore', 'Nepal', 'Malaysia', 'Canada',
       'Australia', 'Cambodia', 'Sri Lanka', 'Germany', 'Finland',
       'United Arab Emirates', 'Philippines', 'India', 'Italy', 'Sweden',
       'Spain', 'Belgium', 'Egypt', 'Lebanon', 'Iraq', 'Oman',
       'Afghanistan', 'Bahrain', 'Kuwait', 'Algeria', 'Croatia',
       'Switzerland', 'Austria', 'Israel', 'Pakistan', 'Brazil',
       'Georgia', 'Greece', 'North Macedonia', 'Norway', 'Romania',
       'Estonia', 'Netherlands', 'San Marino', 'Belarus', 'Iceland',
       'Lithuania', 'Mexico', 'New Zealand', 'Nigeria', 'Ireland',
       'Luxembourg', 'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan',
       'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal',
       'Andorra', 'Latvia', 'Morocco', 'Saudi Arabia', 'Senegal',
       'Argentina', 'Chile', 'Jordan', 'Ukraine', 'Hungary',
       'Liechtenstein', 'Poland', 'Tunisia', 'Bosnia and Herzegovina',
       'Slovenia', 'South Africa', 'Bhutan', 'Cameroon', 'Colo

In [31]:
df1 = df[df['Country/Region'] == country]

In [32]:
df1.shape

(1, 58)

In [33]:
df2 = pd.melt(df1, id_vars = ['Province/State', 'Country/Region', 'Lat', 'Long'], var_name = 'period', value_name = 'case_counts')

In [34]:
df3 = df2[df2['case_counts'] > 0]

In [35]:
df3.shape

(24, 6)

In [36]:
df3.to_csv('Clean_' + country + '_' + filename + '_' + dt + '.csv')

In [37]:
df3.head()

,Province/State,Country/Region,Lat,Long,period,case_counts
30,NaN,Italy,43.0,12.0,2/21/20,1
31,NaN,Italy,43.0,12.0,2/22/20,2
32,NaN,Italy,43.0,12.0,2/23/20,3
33,NaN,Italy,43.0,12.0,2/24/20,7
34,NaN,Italy,43.0,12.0,2/25/20,10


# Local level

In [109]:
zipmap = pd.read_csv('zipcodemap/us-zip-code-latitude-and-longitude.csv', sep = ';')

In [110]:
zipmap.head()

,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


zipcode Lat Long data from
https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/export/


new = zipmap1['geopoint'].str.split(',', n= 1, expand = True)

zipmap1['Long_z'] = new[0]

zipmap1['Lat_z'] = new[1]

In [70]:
zipmap['tmp'] = 1

In [71]:
df3['tmp'] = 1

/Users/hong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


####  join Case with Map

In [72]:
df4 = df3.merge(zipmap, on = 'tmp',how = 'outer')

In [73]:
df4['Country/Region'].value_counts()

US    45523314
Name: Country/Region, dtype: int64

In [74]:
df4.shape

(45523314, 15)

In [75]:
df4.head()

,Province/State,Country/Region,Lat,Long,period,case_counts,tmp,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,71937,Cove,AR,34.398483,-94.39398,-6,1,"34.398483,-94.39398"
1,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,72044,Edgemont,AR,35.624351,-92.16056,-6,1,"35.624351,-92.16056"
2,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,56171,Sherburn,MN,43.660847,-94.74357,-6,1,"43.660847,-94.74357"
3,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,49430,Lamont,MI,43.010337,-85.89754,-5,1,"43.010337,-85.89754"
4,"King County, WA",US,47.6062,-122.3321,1/22/20,1,1,52585,Richland,IA,41.194129,-91.98027,-6,1,"41.194129,-91.98027"


df4['distance'] = math.sqrt(pow(df4['Lat'] - df4['Latitude'], 2) + pow(df4['Long'] - df4['Longitude'], 2))

In [76]:
df4['distance'] = pow(df4['Lat'] - df4['Latitude'], 2) + pow(df4['Long'] - df4['Longitude'], 2)

In [77]:
df5 = df4[df4['distance'] <2.89]

In [78]:
df5.shape

(858188, 16)

In [79]:
df4.shape

(45523314, 16)

In [80]:
df5.to_csv('zipaffected_' + filename + '_' + dt + '.csv')

# National Level

In [38]:
df_national = df3.groupby(['period'])['case_counts'].agg('sum')

In [39]:
df_national.to_csv(country + '_' + filename + "_" + dt + "_national.csv", header = True, index = True)